In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%cd /home/ubuntu/projects/hyper-sloth

In [2]:
from fastcore.all import *
from speedy_utils.all import *
from llm_utils import *

#### Create a LLM model

In [3]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "01"  # Choose any GPU you want to use
from vllm.lora.request import LoRARequest
from vllm import LLM, SamplingParams
import torch

if "llm" in locals():
    del llm  #
    torch.cuda.empty_cache()
    
llm = LLM(
    model="outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merge",
    tensor_parallel_size=4,
    task="generate",
    enforce_eager=True,
    dtype=torch.bfloat16,
    max_model_len=16384,
    enable_lora=False,
    # quantization="bitsandbytes", load_format="bitsandbytes",gpu_memory_utilization=0.95
)

tokenizer = llm.get_tokenizer()

from datasets import load_dataset

# Load the GSM8K dataset
gsm8k = load_dataset("gsm8k", "main")
test = gsm8k["test"]


INFO 03-15 10:50:40 [__init__.py:256] Automatically detected platform cuda.
INFO 03-15 10:50:49 [config.py:1499] Defaulting to use mp for distributed inference
INFO 03-15 10:50:49 [config.py:1677] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 03-15 10:50:49 [cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-15 10:50:51 [core.py:53] Initializing a V1 LLM engine (v0.7.4.dev473+g9ed6ee92) with config: model='outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merge', speculative_config=None, tokenizer='outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merge', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantiza

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorker rank=2 pid=30485) INFO 03-15 10:50:57 [loader.py:429] Loading weights took 0.25 seconds
(VllmWorker rank=1 pid=30464) INFO 03-15 10:50:57 [loader.py:429] Loading weights took 0.27 seconds
(VllmWorker rank=0 pid=30440) INFO 03-15 10:50:57 [loader.py:429] Loading weights took 0.29 seconds
(VllmWorker rank=3 pid=30508) INFO 03-15 10:50:57 [loader.py:429] Loading weights took 0.34 seconds
(VllmWorker rank=0 pid=30440) INFO 03-15 10:50:57 [gpu_model_runner.py:1124] Model loading took 0.7719 GB and 0.736790 seconds
(VllmWorker rank=1 pid=30464) INFO 03-15 10:50:57 [gpu_model_runner.py:1124] Model loading took 0.7719 GB and 0.718700 seconds
(VllmWorker rank=3 pid=30508) INFO 03-15 10:50:57 [gpu_model_runner.py:1124] Model loading took 0.7719 GB and 0.781302 seconds
(VllmWorker rank=2 pid=30485) INFO 03-15 10:50:57 [gpu_model_runner.py:1124] Model loading took 0.7719 GB and 0.692692 seconds
INFO 03-15 10:51:01 [kv_cache_utils.py:537] GPU KV cache size: 1,056,672 tokens
INFO 03-15 1

In [4]:
# Prepare prompts for GSM8K evaluation
all_questions = [item["question"] for item in test][:100]
standardized_prompts = [
    tokenizer.apply_chat_template(
        [{"role": "user", "content": f"{question}\nSolve step by step and put your final numerical answer inside \\boxed{{}}"}],
        tokenize=False,
        add_generation_prompt=True,
    )
    for question in all_questions
]

# Set sampling parameters for deterministic generation
sampling_params = SamplingParams(temperature=0.7, top_p=0.95, top_k=64, max_tokens=10000)

# Generate responses for all questions
outputs = llm.generate(standardized_prompts, sampling_params)
all_outputs = [output.outputs[0].text for output in outputs]


Processed prompts: 100%|██████████| 100/100 [01:41<00:00,  1.02s/it, est. speed input: 101.39 toks/s, output: 452.55 toks/s] 


In [5]:
def get_final_output(response):
    try:
        return int(response.split("\\boxed{")[1].split("}")[0])
    except:
        return None

In [27]:
final_outputs = [get_final_output(response) for response in all_outputs]
accs = []
num_error = 0
for i, gt in enumerate(test):
    if i >= len(final_outputs):
        break
    pred = final_outputs[i]
    try:
        num = gt['answer'].split('####')[1]
        num = int(num)
        pred = int(pred)
        accs.append(num == pred)
    except:
        num_error += 1
        accs.append(0)
        pass

In [29]:
np.mean(accs), num_error/len(final_outputs)

(0.59, 0.1)

In [30]:
len(final_outputs)

100